<a href="https://colab.research.google.com/github/martingkarlssonstudent/SystemAffectiveTouch/blob/main/Sosy_ANN_SL_Touch_Data_Tests_Optimization_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## First we initilize our environment and run it within Tensorflow 2.0

In [ ]:
# Select Tensorflow 2.0
%tensorflow_version 2.x

In [ ]:
# Import needed libraries
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
print('Keras version:',tensorflow.keras.__version__)

import tensorflow.keras as keras

# Helper libraries
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# Matlab plotting
import matplotlib
import matplotlib.pyplot as plt

# importing os module  
import os

from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.keras.layers.experimental import preprocessing

import pandas as pd

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

TensorFlow version: 2.4.1
Keras version: 2.4.0


## Get Data and Massage it into Shape

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
train_data_csv = pd.read_csv("/content/drive/My Drive/ColabData/touchId_3_ALL.csv",sep=',' , decimal=".")
train_data_csv.head()

,0,0.02,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.0366667,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.0533333,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.17.1,0.33,0.34,...,0.2352,0.2353,0.2354,0.2355,0.2356,0.2357,0.2358,0.2359,0.2360,0.2361,0.2362,0.2363,0.2364,0.2365,0.2366,0.2367,0.2368,0.2369,0.2370,0.2371,0.2372,0.2373,0.2374,0.2375,0.2376,0.2377,0.2378,0.2379,0.2380,0.2381,0.2382,0.2383,0.2384,0.2385,0.2386,0.2387,0.2388,0.2389,0.2390,0.2391
0,0,0.000000,0.0,0.0,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
1,0,0.020000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.023333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.04,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.196667,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
2,0,0.000000,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.026667,0.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.026667,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
3,0,0.013333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.020000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.03,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.046667,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
4,0,0.000000,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0


In [ ]:
train_data_0 = np.array(train_data_csv)
train_data_0

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.02     , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.0133333, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.0133333, ..., 0.       , 0.       ,
        0.       ]])

In [ ]:
train_data_0.shape

(1011, 2400)

In [ ]:
train_data_0.dtype

dtype('float64')

In [ ]:
train_labels_csv = pd.read_csv("/content/drive/My Drive/ColabData/touchIdLabel_3_ALL.csv")
train_labels_csv.head()

,1
0,1
1,1
2,1
3,1
4,1


In [ ]:
train_labels_0 = np.array(train_labels_csv)
train_labels_0

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
train_labels_0.shape

(1011, 1)

In [ ]:
train_labels_0.dtype

dtype('int64')

In [ ]:
input_shape = (2400,)

In [ ]:
num_touches = len(np.unique(train_labels_0))
num_touches

10

In [ ]:
tensorflow.keras.utils.to_categorical(train_labels_0).shape

(1011, 10)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(train_data_0, train_labels_0, test_size=0.15)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.15)

print("Shape of train_images: {}".format(train_data.shape))
print("Shape of train_labels: {}".format(train_labels.shape))
print("Shape of test_images: {}".format(test_data.shape))
print("Shape of test_labels: {}".format(test_labels.shape))
print("Shape of val_images: {}".format(val_data.shape))
print("Shape of val_labels: {}".format(val_labels.shape))

Shape of train_images: (1473, 2400)
Shape of train_labels: (1473, 1)
Shape of test_images: (306, 2400)
Shape of test_labels: (306, 1)
Shape of val_images: (261, 2400)
Shape of val_labels: (261, 1)


Create Model

In [ ]:
model = Sequential([
  Flatten(input_shape=input_shape),
#  keras.Input(shape=input_shape),  
  Dense(256, activation='relu'),
  BatchNormalization(),   
  Dense(32, activation='relu'),
  Dropout(0.5),   
  Dense(num_touches, activation='softmax'),
])

# Build the model
# model = Sequential()
# model.add(Flatten(input_shape=(2400,)))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(num_touches, activation='softmax'))

In [ ]:
# Print summary
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 2400)              0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)               614656    
_________________________________________________________________
batch_normalization_11 (Batc (None, 256)               1024      
_________________________________________________________________
dense_34 (Dense)             (None, 32)                8224      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                330       
Total params: 624,234
Trainable params: 623,722
Non-trainable params: 512
_____________________________________________

## Compile and decide on optimizer

In [ ]:
# Compile the model, as a preparation for training
Adam001 = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=Adam001,
              metrics=['accuracy'])

## Training

In [ ]:
to_categorical(train_labels).shape

(1473, 10)

In [ ]:
to_categorical(val_labels).shape

(261, 10)

In [ ]:
epochs = 50      ## Number of epochs to run
batch_size = 32
# Set callback functions to early stop training
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=2, restore_best_weights = True)]

# Train the model.
history = model.fit(
  train_data, 
  to_categorical(train_labels),
  epochs=epochs,
  batch_size=batch_size,
  verbose = 2,
  validation_data=(val_data, to_categorical(val_labels)),
  callbacks=callbacks)

Epoch 1/50
47/47 - 1s - loss: 1.5679 - accuracy: 0.4881 - val_loss: 1.6215 - val_accuracy: 0.3870
Epoch 2/50
47/47 - 0s - loss: 0.9628 - accuracy: 0.6796 - val_loss: 1.3286 - val_accuracy: 0.6169
Epoch 3/50
47/47 - 0s - loss: 0.8182 - accuracy: 0.7210 - val_loss: 1.1127 - val_accuracy: 0.7778
Epoch 4/50
47/47 - 0s - loss: 0.7560 - accuracy: 0.7522 - val_loss: 0.9353 - val_accuracy: 0.7088
Epoch 5/50
47/47 - 0s - loss: 0.6810 - accuracy: 0.7977 - val_loss: 0.8349 - val_accuracy: 0.7701
Epoch 6/50
47/47 - 0s - loss: 0.6633 - accuracy: 0.7977 - val_loss: 0.7217 - val_accuracy: 0.7931
Epoch 7/50
47/47 - 0s - loss: 0.6546 - accuracy: 0.7902 - val_loss: 0.6056 - val_accuracy: 0.8429
Epoch 8/50
47/47 - 0s - loss: 0.5566 - accuracy: 0.8221 - val_loss: 0.5409 - val_accuracy: 0.8889
Epoch 9/50
47/47 - 0s - loss: 0.5126 - accuracy: 0.8330 - val_loss: 0.6693 - val_accuracy: 0.8199
Epoch 10/50
47/47 - 0s - loss: 0.5364 - accuracy: 0.8330 - val_loss: 0.6424 - val_accuracy: 0.8429
Restoring model wei

Evaluation

In [ ]:
score = model.evaluate(test_data, to_categorical(test_labels), verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

Test loss: 0.5320537090301514
Test accuracy: 0.8562091588973999


### Export

In [ ]:
ls /content/drive/

MyDrive/


In [ ]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

------------------------------------------------------------
Frozen model layers: 
x
sequential/flatten/Const
sequential/flatten/Reshape
sequential/dense/MatMul/ReadVariableOp/resource
sequential/dense/MatMul/ReadVariableOp
sequential/dense/MatMul
sequential/dense/BiasAdd/ReadVariableOp/resource
sequential/dense/BiasAdd/ReadVariableOp
sequential/dense/BiasAdd
sequential/dense/Relu
sequential/batch_normalization/batchnorm/ReadVariableOp/resource
sequential/batch_normalization/batchnorm/ReadVariableOp
sequential/batch_normalization/batchnorm/add/y
sequential/batch_normalization/batchnorm/add
sequential/batch_normalization/batchnorm/Rsqrt
sequential/batch_normalization/batchnorm/mul/ReadVariableOp/resource
sequential/batch_normalization/batchnorm/mul/ReadVariableOp
sequential/batch_normalization/batchnorm/mul
sequential/batch_normalization/batchnorm/mul_1
sequential/batch_normalization/batchnorm/ReadVariableOp_2/resource
sequential/batch_normalization/batchnorm/ReadVariableOp_2
sequential

In [ ]:
frozen_out_path = '/content/drive/My Drive/ColabExport'
frozen_graph_filename = 'frozen_graph_SosySLTouch_ALL'
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

'/content/drive/My Drive/ColabExport/frozen_graph_SosySLTouch_ALL.pbtxt'

### Tune

In [ ]:
# Get Keras Tuner
import importlib
try:
    # try to import the module normally and put it in globals
    globals()['kerastuner'] = importlib.import_module('kerastuner')
except ImportError as e:
    !git clone https://github.com/keras-team/keras-tuner.git
    !pip install ./keras-tuner

import kerastuner

# Get some tuner search functions
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.tuners import BayesianOptimization

Cloning into 'keras-tuner'...
remote: Enumerating objects: 6920, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 6920 (delta 82), reused 73 (delta 36), pack-reused 6735
Receiving objects: 100% (6920/6920), 2.20 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (4816/4816), done.
Processing ./keras-tuner
  Created wheel for keras-tuner: filename=keras_tuner-1.0.3-cp37-none-any.whl size=93799 sha256=de8b00a4e0b918ef3f9d2fa1789e190e44a83db08ad0ae954dbab7c4b2d2c308
  Stored in directory: /root/.cache/pip/wheels/fa/4e/d0/ed331a3363786e8a74848aa0589674f502cbb3f0321cdba844
Successfully built keras-tuner


In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=input_shape),
    keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=256, step=16),
        activation='relu'
    ),
    keras.layers.BatchNormalization(),    
    keras.layers.Dense(
        units=hp.Int('dense_2_units', min_value=32, max_value=256, step=16),
        activation='relu'
    ),    
    keras.layers.Dropout(0.5),
    # A Final Dense Layer
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
  return model

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model    

In [ ]:
# To get more stable results we are repeating the runs two times for each parameter setup
MAX_TRIALS = 15         # represents the number of hyperparameter combinations that will be tested by the tuner
EXECUTION_PER_TRIAL = 2 # the number of models that should be built and fit for each trial for robustness purposes

In [ ]:
# To have batch_size as a hyperparameter we need to define our own tuner
# In this case, we are basing it on the BayesianOptimization tuner found in
# https://keras-team.github.io/keras-tuner/documentation/tuners/#bayesianoptimization-class
class MyTuner(kerastuner.tuners.BayesianOptimization):
  def run_trial(self, trial, *args, **kwargs):
    # You can add additional HyperParameters for preprocessing and custom training loops
    # via overriding `run_trial`
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 512, step=32, default=256)
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

In [ ]:
# Bayesian search which also searches for batch_size
tuner = MyTuner(
    build_model,
    max_trials=MAX_TRIALS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='output',
    project_name='SLTouch7' # This name need to be unique for a new tuning session
)

In [ ]:
# Do the hyperparameter search
noepochauto = 9   ## Max number of epochs per trail (but we have early stopping so this max is probably not reached)
tuner.search(train_data, train_labels, 
             epochs=noepochauto, 
             validation_data=(val_data, val_labels), 
             callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)])

Trial 15 Complete [00h 00m 05s]
val_accuracy: 0.7824427485466003

Best val_accuracy So Far: 0.8396946787834167
Total elapsed time: 00h 01m 41s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the best model so far
model = tuner.get_best_models(num_models=1)[0]
# Dump the best hyperparameters found
vals = tuner.get_best_hyperparameters(num_trials=5)[4].values
for keys,values in vals.items():
    print('%20s = %3.4g' %(keys,values))
    
# model.summary()

       dense_1_units = 208
       dense_2_units = 208
       learning_rate = 0.001
          batch_size =  32


##CVS

In [ ]:
clf = svm.SVC(kernel='rbf', C=100, class_weight='balanced', probability=True)

In [ ]:
accuracies = []
confusion_matrices = []

In [ ]:
#TEST_SIZE = 0.15
N_EPOCHS = 10

for k in range(N_EPOCHS):
#      x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=TEST_SIZE, stratify = labels)
      clf.fit(train_data, np.ravel(train_labels,order='C'))
      y_pred = clf.predict(test_data)

      accuracy = metrics.accuracy_score(test_labels, y_pred)
      accuracies.append(accuracy)
      c_matrix = confusion_matrix(test_labels, y_pred)
      confusion_matrices.append(c_matrix)


In [ ]:
print("\nAverage prediction accuracy after " + str(N_EPOCHS) +  " validations:", '{:0.2f}'.format(np.mean(accuracies)))


Average prediction accuracy after 10 validations: 0.85
